In [16]:
import argparse
from lib2to3.pytree import Base
import torch
import numpy as np
import random
import yaml
import gc
import time
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from utils import load_sc_data, load_sc_causal_data, accuracy_LP
from model import CausalGNN
import torchmetrics
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as Data
import sys
from cgi import test
from pyexpat import features
import dgl.data
import networkx as nx 
import scipy.sparse as sp
import scanpy as sc
import pandas as pd


In [17]:
def normalize_features(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx



In [19]:

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape).to_dense()


In [21]:
# load data
data_path = "../example/mESC/ExpressionData.csv"
label_path = "../example/mESC/refNetwork.csv"

In [ ]:
label_file = pd.read_csv(label_path, header = 0, sep = ",")
print(f"label_file : {label_file}")
##gene간의 엣지 연결
data = pd.read_csv(data_path, header = 0, index_col = 0).T
##print(f"data : {data}")
## 세포와 gene간의 expression 파일.
data = data.transform(lambda x: np.log(x + 1))

print(f"data : {data},")

label_file :         Gene1    Gene2
0      ARID1A    KDM3A
1      ARID1A    DIDO1
2      ARID1A     MSH6
3      ARID1A     SFI1
4      ARID1A     RIF1
...       ...      ...
29608    ZIC3    TRP53
29609    ZIC3      VIM
29610    ZIC3  ZFP36L1
29611    ZIC3    ZFP57
29612    ZIC3    ZFP91

[29613 rows x 2 columns]
data :                         BMI1     ARNT2      BTG2      MED4     SOAT1  \
RamDA_mESC_00h_A04  0.557591  0.725131  0.284214  0.939926  0.555779   
RamDA_mESC_00h_A05  0.659003  0.337897  0.409905  0.772341  0.652062   
RamDA_mESC_00h_A06  0.077140  0.734788  0.251473  0.881664  0.347084   
RamDA_mESC_00h_A07  0.369724  0.000000  0.150956  0.767879  0.482680   
RamDA_mESC_00h_A08  0.355905  0.000000  0.000000  0.897754  0.015109   
...                      ...       ...       ...       ...       ...   
RamDA_mESC_72h_H08  0.444515  0.000000  1.111139  0.599446  1.410252   
RamDA_mESC_72h_H09  0.828827  0.563778  0.924926  0.939558  1.551593   
RamDA_mESC_72h_H10  0.550043  

In [ ]:
u = []
v = []
var_names = list(data.columns)
print(f"var_names : {var_names}")
for row_index, row in label_file.iterrows():
    u.append(var_names.index(row[0]))
    v.append(var_names.index(row[1]))
## 각 gene의 연결을 인덱스로만, 숫자로만 u와 v의 리스트에 저장.

print(f"u : {u}")
print(f" v : {v}")

u = np.array(u) ## numpy 배열로 바꿔 하나의 자료형으로 통일하여 효율성 증가. 벡터화 연산 지원.
print(f" np.array: {u}")
u = torch.LongTensor(u)
print(f" torch u : {u}")
v = np.array(v)
v = torch.LongTensor(v)


var_names : ['BMI1', 'ARNT2', 'BTG2', 'MED4', 'SOAT1', 'TLE1', 'NFATC2IP', 'GM13597', 'SOX4', 'YEATS2', 'TXNDC12', 'ODC1', 'GM24044', 'CRAMP1L', 'YAF2', 'SLC25A4', 'GM14303', 'KRBA1', 'MAGED1', 'PDIA6', 'PHLDA3', 'DCXR', 'COL27A1', 'POLR2K', 'FOXP1', 'NOTCH1', 'PHF14', 'SOX15', 'PARVB', 'FOSL2', 'TAF6', 'PURA', 'GJB3', 'SNORD43', 'HNRNPK', 'MYBBP1A', 'CREB3', 'SLC5A11', 'ATF2', 'GM8355', 'E2F6', 'KRT8', 'FOXJ3', 'TCL1', 'FGF10', 'MYMX', 'ARID1A', 'HEY2', 'GM44429', 'IRF4', 'OVOL1', 'SIN3A', 'MIR291B', 'EIF4A2', 'ZFP989', 'NR2F2', 'LDLRAP1', 'BAZ1A', 'MCM2', 'PHF12', 'POU4F2', 'RARG', 'GLMP', 'CUX1', 'FST', 'TEAD3', 'GAS6', 'DPP4', 'GM22862', 'CSRP2', 'KEAP1', 'RNF14', 'PHLDB2', 'KAT6B', 'PER1', 'KAT2B', 'BARD1', 'CRIP2', 'IRF1', 'EDA2R', 'CHD4', 'PATJ', 'ZMYM2', 'POLR3B', 'DNAJC10', 'BEND5', 'FAM129B', 'SLC25A5', 'DMRT1', 'CTSL', 'LAMB2', 'AKR1B8', 'DDX20', 'EED', 'PITX2', 'POLR2E', 'ADAM19', 'KLF13', 'GTF2A1L', 'KLF8', 'IRF3', 'ECT2', 'ZDHHC16', 'RAD54B', 'ABT1', 'AGAP3', 'COIL', 'GM1

In [ ]:
print(label_file.shape) ## gene 갯수 얻기
eids = np.arange(label_file.shape[0])
print(f"eids: {eids}")
eids = np.random.permutation(eids)
print(f" np random eids : {eids}") ## 엣지를 무작위하게 섞어 세트 분배. 10퍼센트 테스트, 10퍼센트 검증.
test_size = int(len(eids) * 0.1)
val_size = test_size
train_size = label_file.shape[0] - test_size - val_size

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
val_pos_u, val_pos_v = u[eids[test_size:(test_size + val_size)]], v[eids[test_size:(test_size + val_size)]]
train_pos_u, train_pos_v = u[eids[(test_size + val_size):]], v[eids[(test_size + val_size):]]
## 그 다음 그 크기에 따라 엣지를 나누어 갖고 있다... 

(29613, 2)
eids: [    0     1     2 ... 29610 29611 29612]
 np random eids : [26146 16200 14586 ...  5712  1524  2605]


In [42]:

    #find all negative edges and split them for training and testing
graph = dgl.graph((u, v), num_nodes = len(var_names))
## gene의 수 만큼 노드 수를 정의하여, u와 v사이 엣지를 만들어준다.
print(graph)


## coo : coordinate 형식. 비어있지 않은 요소들만 저장하는 방식. 
adj = sp.coo_matrix((np.ones(u.shape), (u, v)),
    shape=(len(var_names), len(var_names)),
        dtype=np.float32)
    #adj = graph.adj(scipy_fmt = 'coo')
print(f"u.shape: {u.shape}")
print(f"adj : {adj}")
## (각 엣지의 가중치, (행, 열)), 행렬크기, 데이터 타입으로 인접행렬을 만든다.
adj_neg = 1 - adj.todense() - np.eye(len(var_names))
neg_u, neg_v = np.where(adj_neg != 0)
## negative edge를 찾기 위해 기존 행렬에서 1.1-기존 행렬로 변환. 2. 자기 자신을 제외하기 위해 대각선을 빼기.
## 이후 u와 v의 리스트를 만든다.
print(neg_u) 


Graph(num_nodes=1120, num_edges=29613,
      ndata_schemes={}
      edata_schemes={})
u.shape: torch.Size([29613])
adj :   (46, 119)	1.0
  (46, 723)	1.0
  (46, 147)	1.0
  (46, 187)	1.0
  (46, 559)	1.0
  (46, 685)	1.0
  (46, 724)	1.0
  (46, 752)	1.0
  (46, 418)	1.0
  (46, 507)	1.0
  (46, 750)	1.0
  (46, 793)	1.0
  (46, 758)	1.0
  (46, 751)	1.0
  (46, 236)	1.0
  (46, 540)	1.0
  (46, 976)	1.0
  (46, 473)	1.0
  (46, 249)	1.0
  (46, 1029)	1.0
  (46, 828)	1.0
  (46, 163)	1.0
  (46, 127)	1.0
  (46, 797)	1.0
  (46, 81)	1.0
  :	:
  (314, 300)	1.0
  (314, 286)	1.0
  (314, 404)	1.0
  (314, 78)	1.0
  (314, 495)	1.0
  (314, 653)	1.0
  (314, 217)	1.0
  (314, 720)	1.0
  (314, 1109)	1.0
  (314, 1007)	1.0
  (314, 598)	1.0
  (314, 926)	1.0
  (314, 721)	1.0
  (314, 236)	1.0
  (314, 797)	1.0
  (314, 158)	1.0
  (314, 517)	1.0
  (314, 672)	1.0
  (314, 780)	1.0
  (314, 1088)	1.0
  (314, 140)	1.0
  (314, 336)	1.0
  (314, 924)	1.0
  (314, 899)	1.0
  (314, 746)	1.0
[   0    0    0 ... 1119 1119 1119]


In [ ]:
    ##For 1:1 Pos-Neg
    # neg_eids = np.random.choice(len(neg_u), label_file.shape[0])
    # 
    ##For 1:All Pos-Neg
neg_eids = np.arange(len(neg_u))
print(f"neg_eids_1 : {neg_eids}")
np.random.shuffle(neg_eids)
print(f"neg_eids : {neg_eids}")
## 네거티브 엣지로 쓸 것을 만든다음 랜덤으로 섞음. 이후 데이터 셋 3가지로 분리.

test_neg_size = int(len(neg_eids) * 0.1)
val_neg_size = test_neg_size
train_neg_size = neg_eids.shape[0] - test_neg_size - val_neg_size

test_neg_u, test_neg_v = (
    neg_u[neg_eids[:test_neg_size]],
    neg_v[neg_eids[:test_neg_size]],
    )
val_neg_u, val_neg_v = (
    neg_u[neg_eids[test_neg_size:(test_neg_size + val_neg_size)]],
    neg_v[neg_eids[test_neg_size:(test_neg_size + val_neg_size)]],
    )
train_neg_u, train_neg_v = (
    neg_u[neg_eids[(test_neg_size + val_neg_size):]],
    neg_v[neg_eids[(test_neg_size + val_neg_size):]],
    )


# 세트에따라 포지티브, 네거티브를 합쳐준다. (근데 그냥 합쳐버리면 되는건가? 하긴, 비율만 맞추면 되겠지... 이런식으로 다듬는 방법이 있구나.)

train_u = np.concatenate((train_pos_u, train_neg_u), axis = 0)
train_v = np.concatenate((train_pos_v, train_neg_v), axis = 0)

val_u = np.concatenate((val_pos_u, val_neg_u), axis = 0)
val_v = np.concatenate((val_pos_v, val_neg_v), axis = 0)

test_u = np.concatenate((test_pos_u, test_neg_u), axis = 0)
test_v = np.concatenate((test_pos_v, test_neg_v), axis = 0)


neg_eids_1 : [      0       1       2 ... 1223664 1223665 1223666]
neg_eids : [ 650716  958966 1085247 ...  537372   74312  591397]


In [ ]:
    #Create train and test mask
train_ids = np.stack((train_u, train_v), axis = 1)
train_labels = np.concatenate([np.ones(train_pos_u.shape[0]), np.zeros(train_neg_u.shape[0])], axis = 0)

print(f"train_ids : {train_ids}")
print(f"train_labels : {train_labels}")

val_ids = np.stack((val_u, val_v), axis = 1)
val_labels = np.concatenate([np.ones(val_pos_u.shape[0]), np.zeros(val_neg_u.shape[0])], axis = 0)

test_ids = np.stack((test_u, test_v), axis = 1)
test_labels = np.concatenate([np.ones(test_pos_u.shape[0]), np.zeros(test_neg_u.shape[0])], axis = 0)

## 나란히 붙여서, 각 ... 노드간 연결이 포지티브인지 네거티브인지 확인할 수 있게 _ids와 _labels를 만들었군..



train_ids : [[ 544  802]
 [ 835  573]
 [ 431  254]
 ...
 [ 492   36]
 [  67  338]
 [ 541 1053]]
train_labels : [1. 1. 1. ... 0. 0. 0.]


In [47]:
  
train_g = dgl.remove_edges(graph, eids[:(test_size + val_size)])
    # val_g = dgl.graph((val_pos_u, val_pos_v), num_nodes = len(var_names))
    # test_g = dgl.remove_edges(graph, eids[test_size:])
print(f"train_g : {train_g}")
adj_train = train_g.adj(scipy_fmt = 'coo')
## 앞서 만들었던 그래프에서 test와 validation은 제해준다....
## COO 형식, 좌표 형식으로.. 인접행렬.. 뭘까.
print(f"adj_train : {adj_train}")

train_g : Graph(num_nodes=1120, num_edges=23691,
      ndata_schemes={}
      edata_schemes={})
adj_train :   (46, 119)	1
  (46, 147)	1
  (46, 187)	1
  (46, 559)	1
  (46, 685)	1
  (46, 724)	1
  (46, 752)	1
  (46, 507)	1
  (46, 750)	1
  (46, 793)	1
  (46, 758)	1
  (46, 751)	1
  (46, 236)	1
  (46, 540)	1
  (46, 976)	1
  (46, 473)	1
  (46, 249)	1
  (46, 1029)	1
  (46, 828)	1
  (46, 163)	1
  (46, 127)	1
  (46, 797)	1
  (46, 81)	1
  (46, 979)	1
  (46, 878)	1
  :	:
  (314, 723)	1
  (314, 999)	1
  (314, 870)	1
  (314, 1044)	1
  (314, 707)	1
  (314, 300)	1
  (314, 286)	1
  (314, 404)	1
  (314, 78)	1
  (314, 720)	1
  (314, 1109)	1
  (314, 1007)	1
  (314, 598)	1
  (314, 926)	1
  (314, 721)	1
  (314, 236)	1
  (314, 158)	1
  (314, 517)	1
  (314, 672)	1
  (314, 780)	1
  (314, 1088)	1
  (314, 140)	1
  (314, 924)	1
  (314, 899)	1
  (314, 746)	1


In [ ]:

features = data.T.values
#print(f"data : {data}")
print(f", data.T: {data.T},")
print(f" data.T.values : {data.T.values}")
features = normalize_features(features)
features = torch.FloatTensor(features)
## gene이 각 cell에서 어떻게 나타나는지 하기 위해 행과 열을 바꾸었음.. 

, data.T:          RamDA_mESC_00h_A04  RamDA_mESC_00h_A05  RamDA_mESC_00h_A06  \
BMI1               0.557591            0.659003            0.077140   
ARNT2              0.725131            0.337897            0.734788   
BTG2               0.284214            0.409905            0.251473   
MED4               0.939926            0.772341            0.881664   
SOAT1              0.555779            0.652062            0.347084   
...                     ...                 ...                 ...   
CREB3L2            0.112845            0.297336            0.329994   
ZBTB7A             0.753647            0.550011            0.555693   
MLXIP              0.789515            0.380009            0.339965   
GABBR1             1.237084            1.072264            1.307491   
ATF1               1.296854            1.336010            1.465952   

         RamDA_mESC_00h_A07  RamDA_mESC_00h_A08  RamDA_mESC_00h_A09  \
BMI1               0.369724            0.355905            0.45952

In [ ]:

#Convert the matrix to torch sparse tensor
adj_train = sparse_mx_to_torch_sparse_tensor(adj_train)
train_ids = torch.LongTensor(train_ids)
val_ids = torch.LongTensor(val_ids)
test_ids = torch.LongTensor(test_ids)
train_labels = torch.FloatTensor(train_labels)
val_labels = torch.FloatTensor(val_labels)
test_labels = torch.FloatTensor(test_labels)
## 왜 다시 밀집텐서로 바꾸나 했더니 GPU에선 밀집 텐서가 가장 최적화되어있다고 한다.. 딥러닝 레이어에도 그렇고. 

In [ ]:
# output to a file

# set seed
def set_rng_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

print(sys.argv)


parser = argparse.ArgumentParser()
with open('param.yaml', encoding='utf-8') as f:
    config = yaml.load(f.read(), Loader=yaml.FullLoader)['gcn']
    for key in config.keys():
        name = '--' + key
        parser.add_argument(name, type=type(config[key]), default=config[key])
args = parser.parse_args()

# use cuda
args.cuda = args.cuda and torch.cuda.is_available()
device = torch.device("cuda" if args.cuda else "cpu")

# set seed
set_rng_seed(args.seed)

# load data
data_path = "../example/mESC/ExpressionData.csv"
label_path = "../example/mESC/refNetwork.csv"

print(args.flag)


def load_sc_data(data_path, label_path):
    label_file = pd.read_csv(label_path, header = 0, sep = ",")
    data = pd.read_csv(data_path, header = 0, index_col = 0).T
    data = data.transform(lambda x: np.log(x + 1))

    print(f" label_file : {label_file}, data : {data},")
    u = []
    v = []
    var_names = list(data.columns)
    for row_index, row in label_file.iterrows():
        u.append(var_names.index(row[0]))
        v.append(var_names.index(row[1]))

    u = np.array(u)
    u = torch.LongTensor(u)
    v = np.array(v)
    v = torch.LongTensor(v)

    eids = np.arange(label_file.shape[0])
    eids = np.random.permutation(eids)
    test_size = int(len(eids) * 0.1)
    val_size = test_size
    train_size = label_file.shape[0] - test_size - val_size

    test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
    val_pos_u, val_pos_v = u[eids[test_size:(test_size + val_size)]], v[eids[test_size:(test_size + val_size)]]
    train_pos_u, train_pos_v = u[eids[(test_size + val_size):]], v[eids[(test_size + val_size):]]

    #find all negative edges and split them for training and testing
    graph = dgl.graph((u, v), num_nodes = len(var_names))
    
    adj = sp.coo_matrix((np.ones(u.shape), (u, v)),
                        shape=(len(var_names), len(var_names)),
                        dtype=np.float32)
    #adj = graph.adj(scipy_fmt = 'coo')
    adj_neg = 1 - adj.todense() - np.eye(len(var_names))
    neg_u, neg_v = np.where(adj_neg != 0)

    ##For 1:1 Pos-Neg
    # neg_eids = np.random.choice(len(neg_u), label_file.shape[0])
    # 
    ##For 1:All Pos-Neg
    neg_eids = np.arange(len(neg_u))
    np.random.shuffle(neg_eids)

    test_neg_size = int(len(neg_eids) * 0.1)
    val_neg_size = test_neg_size
    train_neg_size = neg_eids.shape[0] - test_neg_size - val_neg_size

    test_neg_u, test_neg_v = (
        neg_u[neg_eids[:test_neg_size]],
        neg_v[neg_eids[:test_neg_size]],
    )
    val_neg_u, val_neg_v = (
        neg_u[neg_eids[test_neg_size:(test_neg_size + val_neg_size)]],
        neg_v[neg_eids[test_neg_size:(test_neg_size + val_neg_size)]],
    )
    train_neg_u, train_neg_v = (
        neg_u[neg_eids[(test_neg_size + val_neg_size):]],
        neg_v[neg_eids[(test_neg_size + val_neg_size):]],
    )

    train_u = np.concatenate((train_pos_u, train_neg_u), axis = 0)
    train_v = np.concatenate((train_pos_v, train_neg_v), axis = 0)

    val_u = np.concatenate((val_pos_u, val_neg_u), axis = 0)
    val_v = np.concatenate((val_pos_v, val_neg_v), axis = 0)

    test_u = np.concatenate((test_pos_u, test_neg_u), axis = 0)
    test_v = np.concatenate((test_pos_v, test_neg_v), axis = 0)

    #Create train and test mask
    train_ids = np.stack((train_u, train_v), axis = 1)
    train_labels = np.concatenate([np.ones(train_pos_u.shape[0]), np.zeros(train_neg_u.shape[0])], axis = 0)

    val_ids = np.stack((val_u, val_v), axis = 1)
    val_labels = np.concatenate([np.ones(val_pos_u.shape[0]), np.zeros(val_neg_u.shape[0])], axis = 0)

    test_ids = np.stack((test_u, test_v), axis = 1)
    test_labels = np.concatenate([np.ones(test_pos_u.shape[0]), np.zeros(test_neg_u.shape[0])], axis = 0)

    
    train_g = dgl.remove_edges(graph, eids[:(test_size + val_size)])
    # val_g = dgl.graph((val_pos_u, val_pos_v), num_nodes = len(var_names))
    # test_g = dgl.remove_edges(graph, eids[test_size:])
    adj_train = train_g.adj(scipy_fmt = 'coo')


    features = data.T.values
    features = normalize_features(features)
    features = torch.FloatTensor(features)

    #Convert the matrix to torch sparse tensor
    adj_train = sparse_mx_to_torch_sparse_tensor(adj_train)
    train_ids = torch.LongTensor(train_ids)
    val_ids = torch.LongTensor(val_ids)
    test_ids = torch.LongTensor(test_ids)
    train_labels = torch.FloatTensor(train_labels)
    val_labels = torch.FloatTensor(val_labels)
    test_labels = torch.FloatTensor(test_labels)

    return adj_train, features, train_ids, val_ids, test_ids, train_labels, val_labels, test_labels


if args.flag:
    print("load_sc_causal_data")
    adj_train, feature, train_ids, val_ids, test_ids, train_labels, val_labels, test_labels = load_sc_causal_data(data_path, label_path)
else:
    print("load_sc_data")
    adj_train, feature, train_ids, val_ids, test_ids, train_labels, val_labels, test_labels = load_sc_data(data_path, label_path)

print(f"adj_train: {adj_train}")
print(f"feature : {feature}")
print(f"train_ids : {train_ids}")

## 결국 adj train : Graph, feature : gene의 cell별 expression, ids :연결된 엣지의 노드쌍 리스트 labels :포지티브, 네거티브 여부. 
## 어떤식으로 활용하는지 좀 봐야겠...다.  

adj_train = F.normalize(adj_train, p=1, dim=1)

['/home/younggun0816/.local/lib/python3.9/site-packages/ipykernel_launcher.py']
False
load_sc_data
 label_file :         Gene1    Gene2
0      ARID1A    KDM3A
1      ARID1A    DIDO1
2      ARID1A     MSH6
3      ARID1A     SFI1
4      ARID1A     RIF1
...       ...      ...
29608    ZIC3    TRP53
29609    ZIC3      VIM
29610    ZIC3  ZFP36L1
29611    ZIC3    ZFP57
29612    ZIC3    ZFP91

[29613 rows x 2 columns], data :                         BMI1     ARNT2      BTG2      MED4     SOAT1  \
RamDA_mESC_00h_A04  0.557591  0.725131  0.284214  0.939926  0.555779   
RamDA_mESC_00h_A05  0.659003  0.337897  0.409905  0.772341  0.652062   
RamDA_mESC_00h_A06  0.077140  0.734788  0.251473  0.881664  0.347084   
RamDA_mESC_00h_A07  0.369724  0.000000  0.150956  0.767879  0.482680   
RamDA_mESC_00h_A08  0.355905  0.000000  0.000000  0.897754  0.015109   
...                      ...       ...       ...       ...       ...   
RamDA_mESC_72h_H08  0.444515  0.000000  1.111139  0.599446  1.410252   
R